# Analysis for China wage vs. education data

#### Since R cannot be ran on normal VS code, I had to install visual studio code insider for it!

## Datasets used

From Dropbox: China-Education-Wages -> Data -> CFPS Data 2010-2016

Downloaded:
- 2010 Egnlish -> ecfps2010adult_112014.dta     
  - Renamed 2010adult.dta
- 2012 Egnlish -> ecfps2012adultcombine…015.dta     
  - Renamed 2012adult.dta
- 2014 English -> ecfps2014adult_170630.dta     
  - Renamed 2014adult.dta
- Mincer16 -> Mincer16.csv 

All 4 datasets were put within one folder named $CFPSdata$ in the same layer as this jupyter notebook.

In [21]:
library(knitr)
library(xtable)
library(broom)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(stargazer)
library(lubridate)
library(haven)
library(e1071)

ERROR: Error in library(ineq): there is no package called ‘ineq’


## Manipulating data

### Read Data

In [10]:
data10 = read_dta("./CFPSdata/2010adult.dta")
data10 = filter(data10, !qc1 < 0)
data12 = read_dta("./CFPSdata/2012adult.dta")
data12 = filter(data12, !sch2012 < 0)
data14 = read_dta("./CFPSdata/2014adult.dta")
data14 = filter(data14, !pw1r < 0)
data16 = read.csv("./CFPSdata/Mincer16.csv")

### Adding education and year dummies to the data, then combining them into one dataframe

In [15]:
pinc <- function(df) {
    categorize <- 
        df %>%
        group_by(rinc) %>%
        tally()
    categorize = categorize[order(categorize$rinc), ]
    key = categorize$rinc
    pinc = c()
    sum_so_far = 0
    total = nrow(df)
    categorize$pinc = 0
    for (row in 1:nrow(categorize)) {
      sum_so_far = sum_so_far + categorize[row, 'n']
      categorize[row, 'pinc'] = (sum_so_far - (0.5 * categorize[row, 'n'])) / total
    }
    ordered_combined <- df[order(df$rinc), ]
    ordered_combined$pinc = 0
    current_rinc = 0
    current_pinc = 0
    current_row = 0
    for (row in 1:nrow(ordered_combined)) {
      rinc = ordered_combined[row, 'rinc']
      if (rinc != current_rinc) {
        current_row = current_row + 1
        current_rinc = rinc
        current_pinc = categorize[current_row, 'pinc']
      }
      ordered_combined[row, 'pinc'] = current_pinc
    }
    ordered_combined$lnpinc = log(ordered_combined$pinc)
    return(ordered_combined)
}

In [12]:
mincer10 = data.frame(
  income = data10$income, 
  age = data10$qa1age,
  gender = data10$gender,
  urban = data10$urban,
  prov = data10$provcd,
  ethnic = data10$qa5code,
  married = 0,
  party = 0,
  postsecondary = 0,
  seniorsecondary = 0, 
  juniorsecondary = 0,
  primary = 0, 
  illiterate = 0,
  y10 = 1,
  y12 = 0,
  y14 = 0,
  y16 = 0)

for (row in 1:nrow(data10)) {
  marriage = data10[row, "qe1"]
  if (marriage == 2) {
    mincer10[row, "married"] = 1
  }
  party = data10[row, "qa7_s_1"]
  if (party == 1) {
    mincer10[row, "party"] = 1
  }
  edu = data10[row, "qc1"]
  if (edu == 1) {
    mincer10[row, "illiterate"] = 1
  } else if (edu == 2) {
    mincer10[row, "primary"] = 1
  } else if (edu == 3) {
    mincer10[row, "juniorsecondary"] = 1
  } else if (edu == 4) {
    mincer10[row, "seniorsecondary"] = 1
  } else if (edu > 4) {
    mincer10[row, "postsecondary"] = 1
  }
}
mincer10 = filter(mincer10, !is.na(income) & income >= 0)

mincer10_urban = filter(mincer10, urban == 1)
mincer10_rural = filter(mincer10, urban == 0)
mincer10_female_urban = filter(mincer10_urban, gender == 1)
mincer10_female_rural = filter(mincer10_rural, gender == 1)
mincer10_male_urban = filter(mincer10_urban, gender == 0)
mincer10_male_rural = filter(mincer10_rural, gender == 0)

mincer10$rinc = mincer10$income / mean(mincer10$income)

mincer10_urban$rinc = mincer10_urban$income / mean(mincer10_urban$income)
mincer10_rural$rinc = mincer10_rural$income / mean(mincer10_rural$income)
mincer10_female_urban$rinc = mincer10_female_urban$income / mean(mincer10_female_urban$income)
mincer10_female_rural$rinc = mincer10_female_rural$income / mean(mincer10_female_rural$income)
mincer10_male_urban$rinc = mincer10_male_urban$income / mean(mincer10_male_urban$income)
mincer10_male_rural$rinc = mincer10_male_rural$income / mean(mincer10_male_rural$income)

In [13]:
mincer10 <- pinc(mincer10)

mincer10_urban <- pinc(mincer10_urban)

mincer10_rural <- pinc(mincer10_rural)

mincer10_female_urban <- pinc(mincer10_female_urban)

mincer10_female_rural <- pinc(mincer10_female_rural)

mincer10_male_urban <- pinc(mincer10_male_urban)

mincer10_male_rural <- pinc(mincer10_male_rural)

In [14]:
mincer12 = data.frame(
  income = data12$income, 
  age = data12$cfps2012_age,
  gender = data12$cfps2012_gender,
  urban = data12$urban12,
  prov = data12$provcd,
  ethnic = data12$qa701code,
  married = 0,
  party = data12$sn401,
  postsecondary = 0,
  seniorsecondary = 0, 
  juniorsecondary = 0,
  primary = 0, 
  illiterate = 0,
  y10 = 0,
  y12 = 1,
  y14 = 0,
  y16 = 0)

for (row in 1:nrow(data12)) {
  marriage = data12[row, "qe104"]
  if (marriage == 2) {
    mincer12[row, "married"] = 1
  }
  edu = data12[row, "sch2012"]
  if (edu == 1) {
    mincer12[row, "illiterate"] = 1
  } else if (edu == 2) {
    mincer12[row, "primary"] = 1
  } else if (edu == 3) {
    mincer12[row, "juniorsecondary"] = 1
  } else if (edu == 4) {
    mincer12[row, "seniorsecondary"] = 1
  } else if (edu > 4) {
    mincer12[row, "postsecondary"] = 1
  }
}
mincer12 = filter(mincer12, !is.na(income))

mincer12_urban = filter(mincer12, urban == 1)
mincer12_rural = filter(mincer12, urban == 0)
mincer12_female_urban = filter(mincer12_urban, gender == 1)
mincer12_female_rural = filter(mincer12_rural, gender == 1)
mincer12_male_urban = filter(mincer12_urban, gender == 0)
mincer12_male_rural = filter(mincer12_rural, gender == 0)

mincer12$rinc = mincer12$income / mean(mincer12$income)

mincer12_urban$rinc = mincer12_urban$income / mean(mincer12_urban$income)
mincer12_rural$rinc = mincer12_rural$income / mean(mincer12_rural$income)
mincer12_female_urban$rinc = mincer12_female_urban$income / mean(mincer12_female_urban$income)
mincer12_female_rural$rinc = mincer12_female_rural$income / mean(mincer12_female_rural$income)
mincer12_male_urban$rinc = mincer12_male_urban$income / mean(mincer12_male_urban$income)
mincer12_male_rural$rinc = mincer12_male_rural$income / mean(mincer12_male_rural$income)

In [16]:
mincer12 <- pinc(mincer12)

mincer12_urban <- pinc(mincer12_urban)

mincer12_rural <- pinc(mincer12_rural)

mincer12_female_urban <- pinc(mincer12_female_urban)

mincer12_female_rural <- pinc(mincer12_female_rural)

mincer12_male_urban <- pinc(mincer12_male_urban)

mincer12_male_rural <- pinc(mincer12_male_rural)

In [17]:
mincer14 = data.frame(
  income = data14$p_income, 
  age = data14$cfps2014_age,
  gender = data14$cfps_gender,
  urban = data14$urban14,
  prov = data14$provcd14,
  ethnic = data14$cfps_minzu,
  married = 0,
  party = data14$pn401a,
  postsecondary = 0,
  seniorsecondary = 0, 
  juniorsecondary = 0,
  primary = 0, 
  illiterate = 0,
  y10 = 0,
  y12 = 0,
  y14 = 1,
  y16 = 0)

for (row in 1:nrow(data14)) {
  marriage = data14[row, "qea0"]
  if (marriage == 2) {
    mincer14[row, "married"] = 1
  }
  edu = data14[row, "pw1r"]
  if (edu == 1) {
    mincer14[row, "illiterate"] = 1
  } else if (edu == 2) {
    mincer14[row, "primary"] = 1
  } else if (edu == 3) {
    mincer14[row, "juniorsecondary"] = 1
  } else if (edu == 4) {
    mincer14[row, "seniorsecondary"] = 1
  } else if (edu > 4) {
    mincer14[row, "postsecondary"] = 1
  }
}
mincer14 = filter(mincer14, !is.na(income))

mincer14_urban = filter(mincer14, urban == 1)
mincer14_rural = filter(mincer14, urban == 0)
mincer14_female_urban = filter(mincer14_urban, gender == 1)
mincer14_female_rural = filter(mincer14_rural, gender == 1)
mincer14_male_urban = filter(mincer14_urban, gender == 0)
mincer14_male_rural = filter(mincer14_rural, gender == 0)

mincer14$rinc = mincer14$income / mean(mincer14$income)

mincer14_urban$rinc = mincer14_urban$income / mean(mincer14_urban$income)
mincer14_rural$rinc = mincer14_rural$income / mean(mincer14_rural$income)
mincer14_female_urban$rinc = mincer14_female_urban$income / mean(mincer14_female_urban$income)
mincer14_female_rural$rinc = mincer14_female_rural$income / mean(mincer14_female_rural$income)
mincer14_male_urban$rinc = mincer14_male_urban$income / mean(mincer14_male_urban$income)
mincer14_male_rural$rinc = mincer14_male_rural$income / mean(mincer14_male_rural$income)

In [18]:
mincer14 <- pinc(mincer14)

mincer14_urban <- pinc(mincer14_urban)

mincer14_rural <- pinc(mincer14_rural)

mincer14_female_urban <- pinc(mincer14_female_urban)

mincer14_female_rural <- pinc(mincer14_female_rural)

mincer14_male_urban <- pinc(mincer14_male_urban)

mincer14_male_rural <- pinc(mincer14_male_rural)

In [19]:
mincer16 = data.frame(
  income = data16$income, 
  age = data16$age,
  gender = data16$gender,
  urban = data16$urban16,
  prov = data16$provcd16,
  ethnic = data16$ethnic,
  married = data16$married,
  party = data16$party,
  postsecondary = data16$postsecondary,
  seniorsecondary = data16$seniorsecondary, 
  juniorsecondary = data16$juniorsecondary, 
  primary = data16$primary, 
  illiterate = data16$illiterate,
  y10 = 0,
  y12 = 0,
  y14 = 0,
  y16 = 1)
mincer16 = filter(mincer16, !is.na(income))

mincer16_urban = filter(mincer16, urban == 1)
mincer16_rural = filter(mincer16, urban == 0)
mincer16_female_urban = filter(mincer16_urban, gender == 1)
mincer16_female_rural = filter(mincer16_rural, gender == 1)
mincer16_male_urban = filter(mincer16_urban, gender == 0)
mincer16_male_rural = filter(mincer16_rural, gender == 0)

mincer16$rinc = mincer16$income / mean(mincer16$income)

mincer16_urban$rinc = mincer16_urban$income / mean(mincer16_urban$income)
mincer16_rural$rinc = mincer16_rural$income / mean(mincer16_rural$income)
mincer16_female_urban$rinc = mincer16_female_urban$income / mean(mincer16_female_urban$income)
mincer16_female_rural$rinc = mincer16_female_rural$income / mean(mincer16_female_rural$income)
mincer16_male_urban$rinc = mincer16_male_urban$income / mean(mincer16_male_urban$income)
mincer16_male_rural$rinc = mincer16_male_rural$income / mean(mincer16_male_rural$income)

In [20]:
mincer16 <- pinc(mincer16)

mincer16_urban <- pinc(mincer16_urban)

mincer16_rural <- pinc(mincer16_rural)

mincer16_female_urban <- pinc(mincer16_female_urban)

mincer16_female_rural <- pinc(mincer16_female_rural)

mincer16_male_urban <- pinc(mincer16_male_urban)

mincer16_male_rural <- pinc(mincer16_male_rural)

In [ ]:
Gini <- function(df) {
    
}

In [37]:
combined = full_join(mincer10, full_join(mincer12, full_join(mincer14, mincer16)))
combined$lninc = log(combined$income)
combined$lnrinc = log(combined$rinc)

urban = full_join(mincer10_urban, full_join(mincer12_urban, full_join(mincer14_urban, mincer16_urban)))
urban$lninc = log(urban$income)
urban$lnrinc = log(urban$rinc)


rural = full_join(mincer10_rural, full_join(mincer12_rural, full_join(mincer14_rural, mincer16_rural)))
rural$lninc = log(rural$income)
rural$lnrinc = log(rural$rinc)


female_urban = full_join(mincer10_female_urban, full_join(mincer12_female_urban, full_join(mincer14_female_urban, mincer16_female_urban)))
female_urban$lninc = log(female_urban$income)
female_urban$lnrinc = log(female_urban$rinc)


female_rural = full_join(mincer10_female_rural, full_join(mincer12_female_rural, full_join(mincer14_female_rural, mincer16_female_rural)))
female_rural$lninc = log(female_rural$income)
female_rural$lnrinc = log(female_rural$rinc)


male_urban = full_join(mincer10_male_urban, full_join(mincer12_male_urban, full_join(mincer14_male_urban, mincer16_male_urban)))
male_urban$lninc = log(male_urban$income)
male_urban$lnrinc = log(male_urban$rinc)

male_rural = full_join(mincer10_male_rural, full_join(mincer12_male_rural, full_join(mincer14_male_rural, mincer16_male_rural)))
male_rural$lninc = log(male_rural$income)
male_rural$lnrinc = log(male_rural$rinc)

Joining, by = c("income", "age", "gender", "urban", "prov", "ethnic", "married", "party", "postsecondary", "seniorsecondary", "juniorsecondary", "primary", "illiterate", "y10", "y12", "y14", "y16", "rinc")

Joining, by = c("income", "age", "gender", "urban", "prov", "ethnic", "married", "party", "postsecondary", "seniorsecondary", "juniorsecondary", "primary", "illiterate", "y10", "y12", "y14", "y16", "rinc")

Joining, by = c("income", "age", "gender", "urban", "prov", "ethnic", "married", "party", "postsecondary", "seniorsecondary", "juniorsecondary", "primary", "illiterate", "y10", "y12", "y14", "y16", "rinc")

Warning message in .fn(vec_data(.x), ...):
“NaNs produced”
Warning message in log(combined$rinc):
“NaNs produced”
Joining, by = c("income", "age", "gender", "urban", "prov", "ethnic", "married", "party", "postsecondary", "seniorsecondary", "juniorsecondary", "primary", "illiterate", "y10", "y12", "y14", "y16", "rinc")

Joining, by = c("income", "age", "gender", "urban", "prov"

## Running the descriptive analysis for the all of the sub populations

### For combined data

In [10]:
hist(combined$lninc)
hist(urban$lninc)
hist(rural$lninc)
hist(female_urban$lninc)
hist(female_rural$lninc)
hist(male_urban$lninc)
hist(male_rural$lninc)


ERROR: Error in hist(combined$lninc): object 'combined' not found


## Running the regressions for lninc, rinc, lnrinc, pinc, and lnpinc on combined data, and urban and rural subsamples. Does not contain married or party variables

### Regression for lninc

In [82]:
lnincReg = filter(combined, !is.infinite(lninc) & !is.na(lninc))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14, data=lnincReg))

urban_reg = filter(urban, !is.infinite(lninc) & !is.na(lninc))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=urban_reg))

rural_reg = filter(rural, !is.infinite(lninc) & !is.na(lninc))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=rural_reg))

female_urban_reg = filter(female_urban, !is.infinite(lninc) & !is.na(lninc))
female_rural_reg = filter(female_rural, !is.infinite(lninc) & !is.na(lninc))
male_urban_reg = filter(male_urban, !is.infinite(lninc) & !is.na(lninc))
male_rural_reg = filter(male_rural, !is.infinite(lninc) & !is.na(lninc))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_urban_reg))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_rural_reg))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_urban_reg))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_rural_reg))


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = lnincReg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.5493  -0.5264   0.2281   0.8410   5.3336 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      8.797265   0.024534 358.578  < 2e-16 ***
postsecondary    1.732932   0.024212  71.574  < 2e-16 ***
seniorsecondary  1.265809   0.021949  57.670  < 2e-16 ***
juniorsecondary  1.050874   0.018734  56.093  < 2e-16 ***
primary          0.618875   0.020639  29.985  < 2e-16 ***
gender           0.283885   0.009443  30.064  < 2e-16 ***
urban            0.019081   0.005930   3.217  0.00129 ** 
y10             -1.089056   0.021248 -51.255  < 2e-16 ***
y12             -0.446978   0.022283 -20.059  < 2e-16 ***
y14             -1.361638   0.054684 -24.900  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.38


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban_reg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.4944  -0.3971   0.1990   0.6733   4.4564 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      8.99563    0.03223  279.08   <2e-16 ***
postsecondary    1.49882    0.02997   50.02   <2e-16 ***
seniorsecondary  1.05557    0.02905   36.33   <2e-16 ***
juniorsecondary  0.88624    0.02711   32.70   <2e-16 ***
primary          0.55341    0.03122   17.73   <2e-16 ***
gender           0.24503    0.01227   19.96   <2e-16 ***
y10             -0.80633    0.02533  -31.84   <2e-16 ***
y12             -0.26977    0.02643  -10.21   <2e-16 ***
y14             -1.71081    0.07923  -21.59   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.242 on 23150 degrees of freedom
Multiple R-squared:  0.1986,	Adjusted R-squar


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.8980 -0.6364  0.2286  0.8992  5.5796 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      8.85449    0.03690  239.93   <2e-16 ***
postsecondary    1.33277    0.05009   26.61   <2e-16 ***
seniorsecondary  1.04356    0.03552   29.38   <2e-16 ***
juniorsecondary  0.94789    0.02609   36.33   <2e-16 ***
primary          0.56980    0.02716   20.98   <2e-16 ***
gender           0.38078    0.01434   26.55   <2e-16 ***
y10             -1.40198    0.03401  -41.23   <2e-16 ***
y12             -0.66265    0.03585  -18.48   <2e-16 ***
y14             -2.65111    0.09400  -28.20   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.448 on 22284 degrees of freedom
Multiple R-squared:  0.2376,	Adjusted R-squared:  0.237


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_urban_reg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.4420  -0.3562   0.1734   0.6363   4.4840 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      9.48845    0.04226 224.499   <2e-16 ***
postsecondary    1.25608    0.04057  30.958   <2e-16 ***
seniorsecondary  0.81127    0.03938  20.601   <2e-16 ***
juniorsecondary  0.72650    0.03710  19.580   <2e-16 ***
primary          0.35789    0.04175   8.572   <2e-16 ***
y10             -0.73316    0.03114 -23.543   <2e-16 ***
y12             -0.30250    0.03248  -9.314   <2e-16 ***
y14             -1.33373    0.09561 -13.950   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.132 on 12387 degrees of freedom
Multiple R-squared:  0.1498,	Adjusted R-squared:  0.1493 
F-statistic: 311.8 on 7 and 12387 DF,  p-value


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.5980 -0.5356  0.2405  0.8424  5.6423 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      9.36113    0.04409  212.31   <2e-16 ***
postsecondary    1.14427    0.06011   19.04   <2e-16 ***
seniorsecondary  0.94628    0.04176   22.66   <2e-16 ***
juniorsecondary  0.85130    0.03236   26.30   <2e-16 ***
primary          0.48013    0.03405   14.10   <2e-16 ***
y10             -1.31387    0.03962  -33.16   <2e-16 ***
y12             -0.61447    0.04153  -14.80   <2e-16 ***
y14             -2.29392    0.11752  -19.52   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.303 on 12909 degrees of freedom
Multiple R-squared:  0.1981,	Adjusted R-squared:  0.1977 
F-statistic: 455.6 on 7 and 12909 DF,  p-value: < 2.2e-1


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_urban_reg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.4816  -0.3791   0.2467   0.7425   4.3309 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      8.84775    0.04812 183.863  < 2e-16 ***
postsecondary    1.63390    0.04424  36.936  < 2e-16 ***
seniorsecondary  1.18685    0.04296  27.625  < 2e-16 ***
juniorsecondary  0.89749    0.03975  22.577  < 2e-16 ***
primary          0.62488    0.04677  13.361  < 2e-16 ***
y10             -0.87725    0.04052 -21.649  < 2e-16 ***
y12             -0.26280    0.04234  -6.207 5.61e-10 ***
y14             -2.19469    0.12995 -16.888  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.332 on 10693 degrees of freedom
Multiple R-squared:  0.2153,	Adjusted R-squared:  0.2148 
F-statistic: 419.1 on 7 and 10693 DF,  p-value: 


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.8023 -0.7819  0.3052  1.0189  5.3141 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      8.81971    0.06140  143.65   <2e-16 ***
postsecondary    1.48068    0.08454   17.52   <2e-16 ***
seniorsecondary  0.98258    0.06398   15.36   <2e-16 ***
juniorsecondary  0.89573    0.04347   20.60   <2e-16 ***
primary          0.53316    0.04422   12.06   <2e-16 ***
y10             -1.52397    0.05906  -25.80   <2e-16 ***
y12             -0.83170    0.06301  -13.20   <2e-16 ***
y14             -3.00963    0.14965  -20.11   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.593 on 9300 degrees of freedom
Multiple R-squared:  0.1971,	Adjusted R-squared:  0.1965 
F-statistic: 326.2 on 7 and 9300 DF,  p-value: < 2.2e-16


In [83]:
index = c("combined", "urban", "rural", "female_urban", "female_rural", "male_urban", "male_rural")
mean_lninc = c(mean(lnincReg$lninc), mean(urban_reg$lninc), mean(rural_reg$lninc), mean(female_urban_reg$lninc), mean(female_rural_reg$lninc), mean(male_urban_reg$lninc), mean(male_rural_reg$lninc))
median_lninc = c(median(lnincReg$lninc), median(urban_reg$lninc), median(rural_reg$lninc), median(female_urban_reg$lninc), median(female_rural_reg$lninc), median(male_urban_reg$lninc), median(male_rural_reg$lninc))
var_lninc = c(var(lnincReg$lninc), var(urban_reg$lninc), var(rural_reg$lninc), var(female_urban_reg$lninc), var(female_rural_reg$lninc), var(male_urban_reg$lninc), var(male_rural_reg$lninc))
kurtosis_lninc = c(kurtosis(lnincReg$lninc), kurtosis(urban_reg$lninc), kurtosis(rural_reg$lninc), kurtosis(female_urban_reg$lninc), kurtosis(female_rural_reg$lninc), kurtosis(male_urban_reg$lninc), kurtosis(male_rural_reg$lninc))
skew_lninc = c(skewness(lnincReg$lninc), skewness(urban_reg$lninc), skewness(rural_reg$lninc), skewness(female_urban_reg$lninc), skewness(female_rural_reg$lninc), skewness(male_urban_reg$lninc), skewness(male_rural_reg$lninc))

lninc = data.frame("subsamples"=index, "mean" = mean_lninc, "median" = median_lninc, "var" = var_lninc, "kurtosis" = kurtosis_lninc, "skew" = skew_lninc)
lninc$stdev = sqrt(lninc$var)

In [84]:
print("Lninc descriptive statistics")
print(lninc)

[1] "Lninc descriptive statistics"
    subsamples     mean   median      var kurtosis      skew    stdev
1     combined 9.061718 9.392662 2.511959 5.009130 -1.613858 1.584916
2        urban 9.468470 9.784704 1.923729 8.769189 -2.052226 1.386986
3        rural 8.621783 8.922658 2.748698 3.659157 -1.353109 1.657920
4 female_urban 9.714591 9.903488 1.506458 8.792632 -1.866556 1.227378
5 female_rural 9.000477 9.210340 2.116679 4.794995 -1.445970 1.454881
6   male_urban 9.180661 9.510445 2.259995 8.187224 -2.114384 1.503328
7   male_rural 8.096532 8.294050 3.157891 2.919475 -1.226031 1.777046


### Regression for rinc and lnrinc

In [85]:
summary(lm(rinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14, data=combined))
summary(lm(rinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=urban))
summary(lm(rinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=rural))

lnrincReg = filter(combined, !is.infinite(lnrinc) & !is.na(lnrinc))
urban_reg = filter(urban, !is.infinite(lnrinc) & !is.na(lnrinc))
rural_reg = filter(rural, !is.infinite(lnrinc) & !is.na(lnrinc))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14, data=lnrincReg))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=urban_reg))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=rural_reg))


female_urban_reg = filter(female_urban, !is.infinite(lnrinc) & !is.na(lnrinc))
female_rural_reg = filter(female_rural, !is.infinite(lnrinc) & !is.na(lnrinc))
male_urban_reg = filter(male_urban, !is.infinite(lnrinc) & !is.na(lnrinc))
male_rural_reg = filter(male_rural, !is.infinite(lnrinc) & !is.na(lnrinc))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_urban_reg))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_rural_reg))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_urban_reg))
summary(lm(lnrinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_rural_reg))


Call:
lm(formula = rinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = combined)

Residuals:
   Min     1Q Median     3Q    Max 
 -3.07  -0.82  -0.30   0.33 467.73 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     0.145090   0.037147   3.906 9.40e-05 ***
postsecondary   2.432263   0.040480  60.085  < 2e-16 ***
seniorsecondary 0.958014   0.033858  28.295  < 2e-16 ***
juniorsecondary 0.645283   0.027834  23.183  < 2e-16 ***
primary         0.284256   0.030755   9.243  < 2e-16 ***
gender          0.265572   0.014158  18.758  < 2e-16 ***
urban           0.048417   0.009626   5.030 4.92e-07 ***
y10             0.157319   0.034876   4.511 6.47e-06 ***
y12             0.069476   0.034995   1.985   0.0471 *  
y14             0.181437   0.084498   2.147   0.0318 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.829 on 75874 degrees of freedom
Mu


Call:
lm(formula = rinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban)

Residuals:
    Min      1Q  Median      3Q     Max 
 -2.450  -0.743  -0.265   0.380 188.883 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.21874    0.04170   5.245 1.57e-07 ***
postsecondary    1.80552    0.04014  44.985  < 2e-16 ***
seniorsecondary  0.70692    0.03706  19.073  < 2e-16 ***
juniorsecondary  0.46760    0.03383  13.823  < 2e-16 ***
primary          0.21756    0.03923   5.546 2.95e-08 ***
gender           0.27539    0.01648  16.711  < 2e-16 ***
y10              0.08581    0.03592   2.389   0.0169 *  
y12              0.04192    0.03626   1.156   0.2477    
y14              0.15057    0.10123   1.487   0.1369    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.065 on 34502 degrees of freedom
Multiple R-squared:  0.07593,	Adjusted R-squared:  0.07572 



Call:
lm(formula = rinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural)

Residuals:
   Min     1Q Median     3Q    Max 
 -3.19  -0.97  -0.39   0.19 590.63 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.21328    0.06516   3.273  0.00106 ** 
postsecondary    2.45692    0.10378  23.674  < 2e-16 ***
seniorsecondary  1.06649    0.06634  16.075  < 2e-16 ***
juniorsecondary  0.85660    0.04706  18.201  < 2e-16 ***
primary          0.38029    0.04912   7.742    1e-14 ***
gender           0.32190    0.02420  13.299  < 2e-16 ***
y10              0.18160    0.06425   2.826  0.00471 ** 
y12              0.04979    0.06418   0.776  0.43791    
y14              0.19706    0.16908   1.165  0.24383    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.67 on 40329 degrees of freedom
Multiple R-squared:  0.027,	Adjusted R-squared:  0.02681 
F-statistic: 


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = lnrincReg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.5493  -0.5264   0.2281   0.8410   5.3336 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.196444   0.024534 -48.767  < 2e-16 ***
postsecondary    1.732932   0.024212  71.574  < 2e-16 ***
seniorsecondary  1.265809   0.021949  57.670  < 2e-16 ***
juniorsecondary  1.050874   0.018734  56.093  < 2e-16 ***
primary          0.618875   0.020639  29.985  < 2e-16 ***
gender           0.283885   0.009443  30.064  < 2e-16 ***
urban            0.019081   0.005930   3.217  0.00129 ** 
y10             -0.268313   0.021248 -12.628  < 2e-16 ***
y12              0.239660   0.022283  10.755  < 2e-16 ***
y14             -0.711340   0.054684 -13.008  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban_reg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.4944  -0.3971   0.1990   0.6733   4.4564 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.17861    0.03223 -36.565  < 2e-16 ***
postsecondary    1.49882    0.02997  50.017  < 2e-16 ***
seniorsecondary  1.05557    0.02905  36.331  < 2e-16 ***
juniorsecondary  0.88624    0.02711  32.696  < 2e-16 ***
primary          0.55341    0.03122  17.727  < 2e-16 ***
gender           0.24503    0.01227  19.965  < 2e-16 ***
y10             -0.15038    0.02533  -5.937 2.94e-09 ***
y12              0.23186    0.02643   8.772  < 2e-16 ***
y14             -0.85036    0.07923 -10.732  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.242 on 23150 degrees of freedom
Multiple R-squared:  0.157,	Adjusted R-squar


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.8980 -0.6364  0.2286  0.8992  5.5796 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.90691    0.03690 -24.574  < 2e-16 ***
postsecondary    1.33277    0.05009  26.608  < 2e-16 ***
seniorsecondary  1.04356    0.03552  29.379  < 2e-16 ***
juniorsecondary  0.94789    0.02609  36.330  < 2e-16 ***
primary          0.56980    0.02716  20.977  < 2e-16 ***
gender           0.38078    0.01434  26.549  < 2e-16 ***
y10             -0.37282    0.03401 -10.963  < 2e-16 ***
y12              0.25815    0.03585   7.201 6.19e-13 ***
y14             -1.53415    0.09400 -16.321  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.448 on 22284 degrees of freedom
Multiple R-squared:  0.1802,	Adjusted R-squared:  0.17


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_urban_reg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.4420  -0.3562   0.1734   0.6363   4.4840 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.86855    0.04226 -20.550  < 2e-16 ***
postsecondary    1.25608    0.04057  30.958  < 2e-16 ***
seniorsecondary  0.81127    0.03938  20.601  < 2e-16 ***
juniorsecondary  0.72650    0.03710  19.580  < 2e-16 ***
primary          0.35789    0.04175   8.572  < 2e-16 ***
y10             -0.17512    0.03114  -5.623 1.91e-08 ***
y12              0.10011    0.03248   3.083  0.00206 ** 
y14             -0.64754    0.09561  -6.773 1.32e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.132 on 12387 degrees of freedom
Multiple R-squared:  0.1082,	Adjusted R-squared:  0.1077 
F-statistic: 214.6 on 7 and 12387 DF,  p-valu


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.5980 -0.5356  0.2405  0.8424  5.6423 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.68634    0.04409 -15.566  < 2e-16 ***
postsecondary    1.14427    0.06011  19.037  < 2e-16 ***
seniorsecondary  0.94628    0.04176  22.658  < 2e-16 ***
juniorsecondary  0.85130    0.03236  26.304  < 2e-16 ***
primary          0.48013    0.03405  14.101  < 2e-16 ***
y10             -0.34551    0.03962  -8.721  < 2e-16 ***
y12              0.17754    0.04153   4.275 1.92e-05 ***
y14             -1.26564    0.11752 -10.769  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.303 on 12909 degrees of freedom
Multiple R-squared:  0.1263,	Adjusted R-squared:  0.1258 
F-statistic: 266.6 on 7 and 12909 DF,  p-value: < 2.2e-


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_urban_reg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.4816  -0.3791   0.2467   0.7425   4.3309 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.06522    0.04812 -22.136  < 2e-16 ***
postsecondary    1.63390    0.04424  36.936  < 2e-16 ***
seniorsecondary  1.18685    0.04296  27.625  < 2e-16 ***
juniorsecondary  0.89749    0.03975  22.577  < 2e-16 ***
primary          0.62488    0.04677  13.361  < 2e-16 ***
y10             -0.12950    0.04052  -3.196   0.0014 ** 
y12              0.33580    0.04234   7.931 2.39e-15 ***
y14             -0.98714    0.12995  -7.596 3.31e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.332 on 10693 degrees of freedom
Multiple R-squared:  0.1687,	Adjusted R-squared:  0.1682 
F-statistic: 310.1 on 7 and 10693 DF,  p-value:


Call:
lm(formula = lnrinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.8023 -0.7819  0.3052  1.0189  5.3141 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.41054    0.06140  -6.687 2.41e-11 ***
postsecondary    1.48068    0.08454  17.515  < 2e-16 ***
seniorsecondary  0.98258    0.06398  15.358  < 2e-16 ***
juniorsecondary  0.89573    0.04347  20.604  < 2e-16 ***
primary          0.53316    0.04422  12.057  < 2e-16 ***
y10             -0.51795    0.05906  -8.770  < 2e-16 ***
y12              0.24681    0.06301   3.917 9.04e-05 ***
y14             -1.94384    0.14965 -12.990  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.593 on 9300 degrees of freedom
Multiple R-squared:  0.1534,	Adjusted R-squared:  0.1528 
F-statistic: 240.8 on 7 and 9300 DF,  p-value: < 2.2e-16


In [86]:
index = c("combined", "urban", "rural", "female_urban", "female_rural", "male_urban", "male_rural")
mean_lnrinc = c(mean(lnrincReg$lnrinc), mean(urban_reg$lnrinc), mean(rural_reg$lnrinc), mean(female_urban_reg$lnrinc), mean(female_rural_reg$lnrinc), mean(male_urban_reg$lnrinc), mean(male_rural_reg$lnrinc))
median_lnrinc = c(median(lnrincReg$lnrinc), median(urban_reg$lnrinc), median(rural_reg$lnrinc), median(female_urban_reg$lnrinc), median(female_rural_reg$lnrinc), median(male_urban_reg$lnrinc), median(male_rural_reg$lnrinc))
var_lnrinc = c(var(lnincReg$lnrinc), var(urban_reg$lnrinc), var(rural_reg$lnrinc), var(female_urban_reg$lnrinc), var(female_rural_reg$lnrinc), var(male_urban_reg$lnrinc), var(male_rural_reg$lnrinc))
kurtosis_lnrinc = c(kurtosis(lnrincReg$lnrinc), kurtosis(urban_reg$lnrinc), kurtosis(rural_reg$lnrinc), kurtosis(female_urban_reg$lnrinc), kurtosis(female_rural_reg$lnrinc), kurtosis(male_urban_reg$lnrinc), kurtosis(male_rural_reg$lnrinc))
skew_lnrinc = c(skewness(lnrincReg$lnrinc), skewness(urban_reg$lnrinc), skewness(rural_reg$lnrinc), skewness(female_urban_reg$lnrinc), skewness(female_rural_reg$lnrinc), skewness(male_urban_reg$lnrinc), skewness(male_rural_reg$lnrinc))

lnrinc = data.frame("subsamples"=index, "mean" = mean_lnrinc, "median" = median_lnrinc, "var" = var_lnrinc, "kurtosis" = kurtosis_lnrinc, "skew" = skew_lnrinc)
lnrinc$stdev = sqrt(lnrinc$var)

In [87]:
print("Lnrinc descriptive statistics")
print(lnrinc)

[1] "Lnrinc descriptive statistics"
    subsamples       mean        median      var kurtosis      skew    stdev
1     combined -0.2511354  0.0423579142 2.372562 5.466601 -1.682631 1.540312
2        urban -0.1876165  0.0975059813 1.828876 9.320405 -2.110265 1.352360
3        rural -0.2417481  0.0130572746 2.556404 4.085058 -1.425694 1.598876
4 female_urban -0.2135129 -0.0008291416 1.436125 9.551798 -1.948407 1.198384
5 female_rural -0.2321247  0.0258722533 1.942778 5.430126 -1.552104 1.393836
6   male_urban -0.1255579  0.2274507868 2.133439 8.687825 -2.160633 1.460630
7   male_rural -0.2034469  0.0369568992 2.994819 3.249922 -1.277776 1.730555


### Regression for pinc and lnpinc

In [88]:
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14, data=combined))
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=urban))
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=rural))

lnpincReg = filter(combined, !is.infinite(lnpinc) & !is.na(lnpinc))
urban_reg = filter(urban, !is.infinite(lnpinc) & !is.na(lnpinc))
rural_reg = filter(rural, !is.infinite(lnpinc) & !is.na(lnpinc))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14, data=lnpincReg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=rural_reg))


female_urban_reg = filter(female_urban, !is.infinite(lnpinc) & !is.na(lnpinc))
female_rural_reg = filter(female_rural, !is.infinite(lnpinc) & !is.na(lnpinc))
male_urban_reg = filter(male_urban, !is.infinite(lnpinc) & !is.na(lnpinc))
male_rural_reg = filter(male_rural, !is.infinite(lnpinc) & !is.na(lnpinc))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_rural_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_rural_reg))


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = combined)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.77112 -0.21207  0.00658  0.20923  0.95514 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.3737159  0.0034250 109.115  < 2e-16 ***
postsecondary    0.3246912  0.0037323  86.995  < 2e-16 ***
seniorsecondary  0.1759524  0.0031217  56.364  < 2e-16 ***
juniorsecondary  0.1366893  0.0025663  53.263  < 2e-16 ***
primary          0.0755914  0.0028356  26.658  < 2e-16 ***
gender           0.0465633  0.0013053  35.671  < 2e-16 ***
urban            0.0050757  0.0008875   5.719 1.07e-08 ***
y10              0.0318290  0.0032156   9.898  < 2e-16 ***
y12             -0.0526646  0.0032266 -16.322  < 2e-16 ***
y14             -0.0283937  0.0077908  -3.645 0.000268 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard e


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.71825 -0.22090  0.01882  0.21511  0.95824 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.361061   0.005338  67.634  < 2e-16 ***
postsecondary    0.320094   0.005138  62.301  < 2e-16 ***
seniorsecondary  0.169465   0.004745  35.717  < 2e-16 ***
juniorsecondary  0.124701   0.004330  28.796  < 2e-16 ***
primary          0.072042   0.005022  14.346  < 2e-16 ***
gender           0.052831   0.002110  25.044  < 2e-16 ***
y10             -0.001681   0.004598  -0.366    0.715    
y12             -0.041390   0.004642  -8.916  < 2e-16 ***
y14             -0.087746   0.012958  -6.771  1.3e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2644 on 34502 degrees of freedom
Multiple R-squared:  0.1374,	Adjusted R


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.7703 -0.2277 -0.0150  0.2074  0.9637 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.394761   0.004611  85.618  < 2e-16 ***
postsecondary    0.269003   0.007343  36.632  < 2e-16 ***
seniorsecondary  0.137365   0.004694  29.261  < 2e-16 ***
juniorsecondary  0.129111   0.003330  38.770  < 2e-16 ***
primary          0.074323   0.003476  21.383  < 2e-16 ***
gender           0.050609   0.001713  29.550  < 2e-16 ***
y10              0.069736   0.004546  15.339  < 2e-16 ***
y12             -0.060323   0.004541 -13.283  < 2e-16 ***
y14             -0.087683   0.011964  -7.329 2.36e-13 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2597 on 40329 degrees of freedom
Multiple R-squared:  0.1246,	Adjusted R-squared: 


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = lnpincReg)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.1687 -0.5175  0.2313  0.5502  1.9388 

Coefficients:
                 Estimate Std. Error  t value Pr(>|t|)    
(Intercept)     -1.051729   0.010432 -100.816  < 2e-16 ***
postsecondary    0.620287   0.011368   54.563  < 2e-16 ***
seniorsecondary  0.231028   0.009508   24.297  < 2e-16 ***
juniorsecondary  0.193919   0.007817   24.808  < 2e-16 ***
primary          0.120545   0.008637   13.957  < 2e-16 ***
gender           0.101872   0.003976   25.622  < 2e-16 ***
urban           -0.007539   0.002703   -2.789 0.005290 ** 
y10             -0.033663   0.009794   -3.437 0.000589 ***
y12             -0.160249   0.009828  -16.305  < 2e-16 ***
y14             -0.411285   0.023730  -17.332  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.6472 -0.6137  0.2499  0.5613  2.1787 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.129763   0.016719 -67.573  < 2e-16 ***
postsecondary    0.665456   0.016091  41.356  < 2e-16 ***
seniorsecondary  0.284931   0.014860  19.175  < 2e-16 ***
juniorsecondary  0.213510   0.013562  15.743  < 2e-16 ***
primary          0.155830   0.015727   9.908  < 2e-16 ***
gender           0.116298   0.006607  17.603  < 2e-16 ***
y10             -0.097539   0.014401  -6.773 1.28e-11 ***
y12             -0.143339   0.014538  -9.859  < 2e-16 ***
y14             -1.191772   0.040583 -29.366  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8279 on 34502 degrees of freedom
Multiple R-squared:  0.08687,	Adjusted R-sq


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.7594 -0.4507  0.1903  0.5366  2.1768 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.008178   0.013923 -72.410  < 2e-16 ***
postsecondary    0.495768   0.022175  22.357  < 2e-16 ***
seniorsecondary  0.120841   0.014176   8.524  < 2e-16 ***
juniorsecondary  0.169163   0.010056  16.822  < 2e-16 ***
primary          0.105077   0.010496  10.011  < 2e-16 ***
gender           0.106922   0.005172  20.674  < 2e-16 ***
y10              0.056534   0.013729   4.118 3.83e-05 ***
y12             -0.154704   0.013714 -11.281  < 2e-16 ***
y14             -1.281240   0.036127 -35.464  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7842 on 40329 degrees of freedom
Multiple R-squared:  0.07212,	Adjusted R-sq


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.0630 -0.6804  0.2644  0.5851  1.8740 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.13257    0.02669 -42.436  < 2e-16 ***
postsecondary    0.65362    0.02628  24.873  < 2e-16 ***
seniorsecondary  0.27704    0.02478  11.178  < 2e-16 ***
juniorsecondary  0.23055    0.02319   9.942  < 2e-16 ***
primary          0.15377    0.02625   5.857 4.80e-09 ***
y10             -0.07076    0.02093  -3.380 0.000727 ***
y12             -0.13432    0.02116  -6.347 2.25e-10 ***
y14             -0.91680    0.05922 -15.481  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8698 on 16734 degrees of freedom
Multiple R-squared:  0.05728,	Adjusted R-squared:  0.05689 
F-statistic: 145.3 on 7 and 16734 DF,  p-value: < 2.


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.2647 -0.6055  0.1916  0.5865  2.0517 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.08738    0.02078 -52.337  < 2e-16 ***
postsecondary    0.51258    0.03207  15.981  < 2e-16 ***
seniorsecondary  0.16127    0.02069   7.796 6.72e-15 ***
juniorsecondary  0.23594    0.01592  14.817  < 2e-16 ***
primary          0.13626    0.01683   8.095 6.03e-16 ***
y10              0.11675    0.01973   5.916 3.34e-09 ***
y12             -0.10742    0.01973  -5.443 5.29e-08 ***
y14             -0.99675    0.05606 -17.779  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.831 on 20008 degrees of freedom
Multiple R-squared:  0.04529,	Adjusted R-squared:  0.04496 
F-statistic: 135.6 on 7 and 20008 DF,  p-value: < 2.2


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.1326 -0.5651  0.2363  0.5443  2.4341 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.98554    0.02180 -45.200  < 2e-16 ***
postsecondary    0.65136    0.02096  31.079  < 2e-16 ***
seniorsecondary  0.24861    0.01901  13.080  < 2e-16 ***
juniorsecondary  0.15600    0.01697   9.192  < 2e-16 ***
primary          0.11805    0.01993   5.924 3.19e-09 ***
y10             -0.12056    0.02013  -5.990 2.14e-09 ***
y12             -0.16752    0.02029  -8.258  < 2e-16 ***
y14             -1.49222    0.05641 -26.453  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7982 on 17659 degrees of freedom
Multiple R-squared:  0.09479,	Adjusted R-squared:  0.09443 
F-statistic: 264.2 on 7 and 17659 DF,  p-value: < 2.2e


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.8795 -0.2963 -0.2355  0.5215  2.2991 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.810848   0.019111 -42.429  < 2e-16 ***
postsecondary    0.464851   0.031836  14.601  < 2e-16 ***
seniorsecondary  0.034703   0.020934   1.658  0.09739 .  
juniorsecondary  0.060803   0.013582   4.477 7.62e-06 ***
primary          0.044728   0.013795   3.242  0.00119 ** 
y10             -0.006346   0.019494  -0.326  0.74478    
y12             -0.221887   0.019454 -11.406  < 2e-16 ***
y14             -1.531086   0.047536 -32.209  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7509 on 20125 degrees of freedom
Multiple R-squared:  0.07499,	Adjusted R-squared:  0.07467 
F-statistic: 233.1 on 7 and 20125 DF,  p-valu

In [89]:
index = c("combined", "urban", "rural", "female_urban", "female_rural", "male_urban", "male_rural")
mean_lnpinc = c(mean(lnpincReg$lnpinc), mean(urban_reg$lnpinc), mean(rural_reg$lnpinc), mean(female_urban_reg$lnpinc), mean(female_rural_reg$lnpinc), mean(male_urban_reg$lnpinc), mean(male_rural_reg$lnpinc))
median_lnpinc = c(median(lnpincReg$lnpinc), median(urban_reg$lnpinc), median(rural_reg$lnpinc), median(female_urban_reg$lnpinc), median(female_rural_reg$lnpinc), median(male_urban_reg$lnpinc), median(male_rural_reg$lnpinc))
var_lnpinc = c(var(lnpincReg$lnpinc), var(urban_reg$lnpinc), var(rural_reg$lnpinc), var(female_urban_reg$lnpinc), var(female_rural_reg$lnpinc), var(male_urban_reg$lnpinc), var(male_rural_reg$lnpinc))
kurtosis_lnpinc = c(kurtosis(lnpincReg$lnpinc), kurtosis(urban_reg$lnpinc), kurtosis(rural_reg$lnpinc), kurtosis(female_urban_reg$lnpinc), kurtosis(female_rural_reg$lnpinc), kurtosis(male_urban_reg$lnpinc), kurtosis(male_rural_reg$lnpinc))
skew_lnpinc = c(skewness(lnpincReg$lnpinc), skewness(urban_reg$lnpinc), skewness(rural_reg$lnpinc), skewness(female_urban_reg$lnpinc), skewness(female_rural_reg$lnpinc), skewness(male_urban_reg$lnpinc), skewness(male_rural_reg$lnpinc))

lnpinc = data.frame("subsamples"=index, "mean" = mean_lnpinc, "median" = median_lnpinc, "var" = var_lnpinc, "kurtosis" = kurtosis_lnpinc, "skew" = skew_lnpinc)
lnpinc$stdev = sqrt(lnpinc$var)

In [90]:
print("Lnpinc descriptive statistics")
print(lnpinc)

[1] "Lnpinc descriptive statistics"
    subsamples       mean     median       var  kurtosis      skew     stdev
1     combined -0.9264864 -0.6932131 0.6700671  5.953531 -1.864717 0.8185763
2        urban -0.9452581 -0.6941909 0.7504527  7.271397 -2.003249 0.8662867
3        rural -0.9151351 -0.6930976 0.6626776 10.318108 -2.343026 0.8140501
4 female_urban -0.9611053 -0.6867171 0.8022726  5.948998 -1.864653 0.8956967
5 female_rural -0.9375308 -0.6975534 0.7231072  7.971376 -2.057542 0.8503571
6   male_urban -0.9293849 -0.6909986 0.7035739  8.746663 -2.172646 0.8387931
7   male_rural -0.8926620 -1.2682299 0.6092807 12.960143 -2.692035 0.7805643
